In [ ]:
pip install "pycozo[embedded,pandas]"

In [ ]:
from pycozo.client import Client
import pandas as pd
from pycozo.client import Client
client = Client('rocksdb', 'Benchmark_cozo.db')

In [ ]:
#transform cozo nodes + prop to csv
# this code is used to transform only 
import os
import pandas as pd

# 1. Charger les dataframes depuis CozoDB
node_label_df = pd.DataFrame(client.run('?[id_n,ln] := *node_label[id_n,ln]'))
node_label_df = node_label_df[(node_label_df["id_n"] >= 2900001) &(node_label_df["id_n"] <= 3000000)]


node_prop_df  = pd.DataFrame(client.run('?[id_n,key,value] := *node_prop[id_n,key,value]'))
node_prop_df = node_prop_df[(node_prop_df["id_n"] >= 2900001) &(node_prop_df["id_n"] <= 3000000)]



# 2) Préparer le répertoire de sortie
output_dir = "nodes_csv"
os.makedirs(output_dir, exist_ok=True)

# 3) Pour chaque label, construire et écrire le CSV dans ce répertoire
for label in node_label_df['ln'].unique():
    # IDs pour ce label
    ids = node_label_df.loc[node_label_df['ln'] == label, 'id_n']
    
    # Filtrer les propriétés
    props = node_prop_df[node_prop_df['id_n'].isin(ids)]
    
    # DataFrame de base avec tous les IDs
    base = pd.DataFrame({'id_n': ids}).drop_duplicates().set_index('id_n')
    
    # Pivoter et joindre si des propriétés existent
    if not props.empty:
        pivot = props.pivot(index='id_n', columns='key', values='value')
        df = base.join(pivot, how='left')
    else:
        df = base.copy()
    
    # Remettre id_n en colonne
    df = df.reset_index()
    
    # Écrire le fichier CSV dans le répertoire
    filename = f"{label.upper()}.csv"
    path = os.path.join(output_dir, filename)
    df.to_csv(path, index=False, encoding='utf-8')

print("c'est done capitaine pour nodes!")



In [ ]:
#transform edge_prop to csv
# DO NOT FORGET TO UPDATE the start and end point 
import os
import pandas as pd

# Récupérer les données depuis CozoDB
edge_label = pd.DataFrame(client.run('?[id_e, ln] := *edge_label[id_e, ln]'))
edge_label_df = edge_label[(edge_label["id_e"] >= 16_630_881) &(edge_label["id_e"] <= 16_631_282)]

edge = pd.DataFrame(client.run('?[id_e, ns, nd] := *edge[id_e, ns, nd]'))
edge_df = edge[(edge["id_e"] >= 16_630_881) &(edge["id_e"] <= 16_631_282)]


edge_prop = pd.DataFrame(client.run('?[id_e, key, value] := *edge_prop[id_e, key, value]'))
edge_prop_df = edge_prop[(edge_prop["id_e"] >= 16_630_881) &(edge_prop["id_e"] <= 16_631_282)]

# 2) Préparer le répertoire de sortie
output_dir = "edges_csv"
os.makedirs(output_dir, exist_ok=True)

# 3) Pour chaque label, construire et écrire le CSV dans ce répertoire
for label in edge_label_df['ln'].unique():
    # 3.1) Tous les id_e pour ce label
    ids = edge_label_df.loc[edge_label_df['ln'] == label, 'id_e']
    
    # 3.2) Extraire from/to pour ces id_e
    base = edge_df[edge_df['id_e'].isin(ids)].rename(columns={'ns':'from_id','nd':'to_id'})
    base = base.set_index('id_e')
    
    # 3.3) Filtrer les propriétés de ces id_e
    props = edge_prop_df[edge_prop_df['id_e'].isin(ids)]
    
    # 3.4) Pivoter et joindre si des propriétés existent
    if not props.empty:
        pivot = props.pivot(index='id_e', columns='key', values='value')
        df = base.join(pivot, how='left')
    else:
        df = base.copy()
    
    # 3.5) Remettre id_e en colonne
    df = df.reset_index()
    
    # 3.6) Écrire le fichier CSV (nom = label en minuscules)
    filename = f"{label}.csv"
    path = os.path.join(output_dir, filename)
    df.to_csv(path, index=False, encoding='utf-8')

print("edges are done capitain !")
